In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from copy import deepcopy
from scipy.sparse.csgraph import laplacian
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from scipy.sparse.csgraph import laplacian

pp = dict(surface='white', hemi='both')

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape
dist = cdist(pos, pos)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.7s remaining:    1.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished


In [3]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=1, extents=(1, 40), duration_of_trial=0.001, target_snr=99999)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
y = evoked.data
x = stc.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 401.08it/s]


Using pyvistaqt 3d backend.

Using control points [0.0000000e+00 0.0000000e+00 1.2234291e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


## Test Generator

In [5]:
from invert.solvers import generator

gen = generator(fwd, "cov", batch_size=1, n_sources=4, n_timepoints=20, snr_range=(100, 101))
x_test, y_test = gen.__next__()
print(x_test.shape, y_test.shape)
i = 0
%matplotlib qt
evoked_ = mne.EvokedArray(x_test[i, :, :, 0].T, evoked.info)
evoked_.plot_joint(title="Sample")

stc_ = stc.copy()

stc_.data[:, 0] = y_test[i]
stc_.plot(**pp)

(1, 64, 64, 1) (1, 1284)
No projector specified for this dataset. Please consider the method self.add_proj.
Using control points [0. 0. 1.]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Training

## CNN

In [55]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, BatchNormalization
from tensorflow.keras import backend as K
tf.keras.backend.set_image_data_format('channels_last')

n_channels = evoked.data.shape[0]
n_dipoles = x.shape[0]

# Architecture Params
n_filters = 256
activation_function = "tanh"
batch_size = int(n_dipoles)

# Simulation Params
n_sources = 10
n_orders = 2
n_timepoints = 20
batch_repetitions = 10
snr_range = (1e99,2e99)
amplitude_range = (0.99,1)
gen_args = dict(batch_size=batch_size, batch_repetitions=batch_repetitions, 
                n_sources=n_sources, n_orders=n_orders, n_timepoints=n_timepoints,
                snr_range=snr_range, amplitude_range=amplitude_range)


# Training Params
epochs = 300
steps_per_epoch = batch_repetitions

n_hl = 1

inputs = tf.keras.Input(shape=(n_channels, n_channels, 1), name='Input')

cnn1 = Conv2D(n_filters, (1, n_channels),
            activation=activation_function, padding="valid",
            name='CNN1')(inputs)
# cnn1 = BatchNormalization()(cnn1)
# cnn1 = Conv2D(n_filters, (n_channels, 1),
#             activation=activation_function, padding="valid",
#             name='CNN2')(cnn1)




flat = Flatten()(cnn1)

fc1 = Dense(300, 
            activation=activation_function, 
            name='FC1')(flat)
# fc1 = Dense(300, 
#             activation=activation_function, 
#             name='FC2')(fc1)

out = Dense(n_dipoles, 
            activation="sigmoid", 
            name='Output')(fc1)


# model = tf.keras.Model(inputs=inputs, outputs=out, name='CovCNN')
# model.compile(loss="cosine_similarity", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))
# model.summary()

gen = generator(fwd, "cov", **gen_args)
gen_args["batch_size"] = 1024
gen_val = generator(fwd, "cov", **gen_args)
callbacks = [tf.keras.callbacks.EarlyStopping(patience=epochs, restore_best_weights=True),]
model.fit(x=gen, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=gen_val.__next__(), callbacks=callbacks)


Epoch 1/300
10/10 [==============================] - 1s 101ms/step - loss: -0.2988 - val_loss: -0.6092
Epoch 2/300
10/10 [==============================] - 2s 52ms/step - loss: -0.3093 - val_loss: -0.6065
Epoch 3/300
10/10 [==============================] - 2s 60ms/step - loss: -0.3037 - val_loss: -0.6037
Epoch 4/300
10/10 [==============================] - 2s 51ms/step - loss: -0.3117 - val_loss: -0.6002
Epoch 5/300
10/10 [==============================] - 2s 50ms/step - loss: -0.3098 - val_loss: -0.5888
Epoch 6/300
10/10 [==============================] - 2s 49ms/step - loss: -0.3215 - val_loss: -0.5803
Epoch 7/300
10/10 [==============================] - 2s 42ms/step - loss: -0.3228 - val_loss: -0.5690
Epoch 8/300
10/10 [==============================] - 2s 39ms/step - loss: -0.3384 - val_loss: -0.5671
Epoch 9/300
10/10 [==============================] - 2s 37ms/step - loss: -0.3404 - val_loss: -0.5610
Epoch 10/300
10/10 [==============================] - 1s 37ms/step - loss: -0.340

KeyboardInterrupt: 

# Eval

In [56]:
clim = dict(kind="percent", lims=(0,0.5,1))

gen_args = dict(batch_size=batch_size, batch_repetitions=batch_repetitions, 
                n_sources=n_sources, n_orders=n_orders, n_timepoints=n_timepoints)
gen_args["batch_size"] = 10
gen_args["amplitude_range"] = (0.99, 1)

gen_tst = generator(fwd, "cov", **gen_args)

x_test, y_test = gen_tst.__next__()

y_hat = model.predict(x_test, verbose=0)

stc_ = stc.copy()
stc_.data = (y_test.T / y_test.max( axis=1))
stc_.plot(**pp, brain_kwargs=dict(title="Ground Truths"), colormap="Reds", clim=clim)

stc_ = stc.copy()
stc_.data = (y_hat.T / y_hat.max( axis=1))
stc_.plot(**pp, brain_kwargs=dict(title="Preds"), colormap="Reds", clim=clim)



Using control points [0. 0. 0.]


C:\Users\Lukas\AppData\Local\Temp\ipykernel_11324\2140372095.py:16: RuntimeWarning: All data were zero
  stc_.plot(**pp, brain_kwargs=dict(title="Ground Truths"), colormap="Reds", clim=clim)


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using control points [4.52346001e-07 6.53062199e-06 1.12960853e-05]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [0. 0. 0.]


Traceback (most recent call last):
  File "c:\Users\Lukas\Envs\invertenv\lib\site-packages\mne\viz\utils.py", line 60, in safe_event
    return fun(*args, **kwargs)
  File "c:\Users\Lukas\Envs\invertenv\lib\site-packages\mne\viz\_brain\_brain.py", line 731, in _clean
    self.clear_glyphs()
  File "c:\Users\Lukas\Envs\invertenv\lib\site-packages\mne\viz\_brain\_brain.py", line 1629, in clear_glyphs
    assert sum(len(v) for v in self.picked_points.values()) == 0
AssertionError


In [42]:
y_test.shape

(10, 1284)

In [43]:
y_hat.shape

(10, 1284)